LinearRegression_Phase5_การทดลองที่2_120Train, lag_7, timebase, sales_gb

#Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv")
df.head()

In [ ]:
df.columns

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

In [ ]:
df['is_weekend'] = (df.index.dayofweek == 6).astype(int)
df.head()

ตัวแปรที่ไม่ได้มีความหมายเชิงลำดับเช่น dayofweek, month  
เพราะตัวแปรเหล่านี้เป็นข้อมูลเชิงกลุ่ม (Categorical) ไม่ใช่เชิงลำดับ (Ordinal) หากเราใส่เป็นตัวเลข 1-12 เข้าไปตรงๆ โมเดลจะเข้าใจผิดว่า "เดือนธันวาคม (12)" มีค่ามากกว่า "เดือนมกราคม (1)" ถึง 12 เท่า ซึ่งไม่เป็นความจริง การทำ One-Hot Encoding จะสร้างคอลัมน์ใหม่เป็น is_Jan, is_Feb,... เพื่อแก้ปัญหานี้

In [ ]:
# prompt: สร้าง 'is_Sun' จาก dayofweek ถ้าตรงกับวันอาทิตย์แสดงเป็น 1  ถ้าไม่ตรงแสดงเป็น 0

df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df.head(5)

In [ ]:
df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df['is_Mon'] = (df.index.dayofweek == 0).astype(int)
df['is_Tue'] = (df.index.dayofweek == 1).astype(int)
df['is_Wed'] = (df.index.dayofweek == 2).astype(int)
df['is_Thu'] = (df.index.dayofweek == 3).astype(int)
df['is_Fri'] = (df.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df['is_Jan'] = (df.index.month == 1).astype(int)
df['is_Feb'] = (df.index.month == 2).astype(int)
df['is_Mar'] = (df.index.month == 3).astype(int)
df['is_Apr'] = (df.index.month == 4).astype(int)
df['is_May'] = (df.index.month == 5).astype(int)
df['is_Jun'] = (df.index.month == 6).astype(int)
df['is_Jul'] = (df.index.month == 7).astype(int)
df['is_Aug'] = (df.index.month == 8).astype(int)
df['is_Sep'] = (df.index.month == 9).astype(int)
df['is_Oct'] = (df.index.month == 10).astype(int)
df['is_Nov'] = (df.index.month == 11).astype(int)
df['is_Dec'] = (df.index.month == 12).astype(int)
df.head(3)

ในการทดลอง Phase3 จะใช้แค่ 'sales_sum', 'time_index', 'lag_7' และตัวแปรทางปฏิทิน

In [ ]:
df['sales_fs_lag_1'] = df['sales_fs'].shift(1)
df['sales_fs_lag_2'] = df['sales_fs'].shift(2)
df['sales_fs_lag_3'] = df['sales_fs'].shift(3)
df['sales_fs_lag_4'] = df['sales_fs'].shift(4)
df['sales_fs_lag_5'] = df['sales_fs'].shift(5)
df['sales_fs_lag_6'] = df['sales_fs'].shift(6)
df['sales_fs_lag_7'] = df['sales_fs'].shift(7)

In [ ]:
df['sales_gb_lag_1'] = df['sales_gb'].shift(1)
df['sales_gb_lag_2'] = df['sales_gb'].shift(2)
df['sales_gb_lag_3'] = df['sales_gb'].shift(3)
df['sales_gb_lag_4'] = df['sales_gb'].shift(4)
df['sales_gb_lag_5'] = df['sales_gb'].shift(5)
df['sales_gb_lag_6'] = df['sales_gb'].shift(6)
df['sales_gb_lag_7'] = df['sales_gb'].shift(7)

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df = df.reindex(columns=['sales_sum', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                         , 'year', 'time_index'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'sales_fs_lag_1', 'sales_fs_lag_2', 'sales_fs_lag_3', 'sales_fs_lag_4', 'sales_fs_lag_5', 'sales_fs_lag_6', 'sales_fs_lag_7'
                         , 'sales_gb_lag_1', 'sales_gb_lag_2', 'sales_gb_lag_3', 'sales_gb_lag_4', 'sales_gb_lag_5', 'sales_gb_lag_6', 'sales_gb_lag_7'
                         ])

df.head()

In [ ]:
# prompt: drop row that have missing value

df.dropna(inplace=True)
df.head()

In [ ]:
df_last_127 = df.tail(127)
df_last_127

In [ ]:
# prompt: แบ่ง test_data = 7 row ท้ายสุด,  train_data = row ทั้งหมดที่เหลือ

test_data = df_last_127.tail(7)
train_data = df_last_127.head(120)

print("Test Data:")
print(test_data)

print("\nTrain Data:")
train_data

In [ ]:
X = train_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                         , 'year', 'time_index'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'sales_fs_lag_1', 'sales_fs_lag_2', 'sales_fs_lag_3', 'sales_fs_lag_4', 'sales_fs_lag_5', 'sales_fs_lag_6', 'sales_fs_lag_7'
                         , 'sales_gb_lag_1', 'sales_gb_lag_2', 'sales_gb_lag_3', 'sales_gb_lag_4', 'sales_gb_lag_5', 'sales_gb_lag_6', 'sales_gb_lag_7'
                ]]
y = train_data['sales_sum']

#Train Model

In [ ]:
# Initializing the linear regression model
model = LinearRegression()

# Training the model on the training data available
model.fit(X, y)

In [ ]:
X_test = test_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                          , 'year', 'time_index'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'sales_fs_lag_1', 'sales_fs_lag_2', 'sales_fs_lag_3', 'sales_fs_lag_4', 'sales_fs_lag_5', 'sales_fs_lag_6', 'sales_fs_lag_7'
                         , 'sales_gb_lag_1', 'sales_gb_lag_2', 'sales_gb_lag_3', 'sales_gb_lag_4', 'sales_gb_lag_5', 'sales_gb_lag_6', 'sales_gb_lag_7'
                    ]]
predictions = model.predict(X_test)

# Print the predictions
print("\nPredictions:")
predictions

#Evaluate

In [ ]:
# Calculating MSE
mse = mean_squared_error(test_data['sales_sum'], predictions)
print(f"\nMean Squared Error (MSE): {mse}")

# Calculating RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")